In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Cargar el modelo GPT-4 y el tokenizador
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Cargar los datos de películas desde el archivo CSV
movies_data = pd.read_csv("/content/drive/MyDrive/dataIA/movies/movies_with_langKeywRevised.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# GPT
Se ha observado que se encuentran listas vacías en los géneros de las películas por lo que se procede a usar GPT para que genere estos géneros a partir de las sinopsis.

In [3]:
movies_data.head()

,title,genres_associated,languages_availables,runtime,overview,adult,release_date,tagline,vote_average,keywords,cast
0,Jumanji,"['Adventure', 'Fantasy', 'Family']","['English', 'French']",104.0,When siblings Judy and Peter discover an encha...,False,1995-12-15,Roll the dice and unleash the excitement!,6.9,"['game', 'alan', 'siblings']","['Robin Williams', 'Jonathan Hyde', 'Kirsten D..."
1,Grumpier Old Men,"['Romance', 'Comedy']",['English'],101.0,A family wedding reignites the ancient feud be...,False,1995-12-22,Still Yelling. Still Fighting. Still Ready for...,6.5,"['max', 'family', 'wedding']","['Walter Matthau', 'Jack Lemmon', 'Ann-Margret..."
2,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",['English'],127.0,"Cheated on, mistreated and stepped on, the wom...",False,1995-12-22,Friends are the people who let you be yourself...,6.1,"['cheated', 'mistreated', 'stepped']","['Whitney Houston', 'Angela Bassett', 'Loretta..."
3,Father of the Bride Part II,['Comedy'],['English'],106.0,Just when George Banks has recovered from his ...,False,1995-02-10,Just When His World Is Back To Normal... He's ...,5.7,"['george', 'banks', 'recovered']","['Steve Martin', 'Diane Keaton', 'Martin Short..."
4,Heat,"['Action', 'Crime', 'Drama', 'Thriller']","['English', 'Spanish']",170.0,"Obsessive master thief, Neil McCauley leads a ...",False,1995-12-15,A Los Angeles Crime Saga,7.7,"['obsessive', 'master', 'thief']","['Al Pacino', 'Robert De Niro', 'Val Kilmer', ..."


In [4]:
import re

for index, row in movies_data.iterrows():
    if row['genres_associated'] == '[]':
        print("Film: ", row['title'])
        description = row['overview']
        # Construir el prompt para guiar al modelo a generar géneros basados en la descripción
        prompt = f"Description: {description}. Genres:"
        # Codificar el prompt utilizando el tokenizador del modelo
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        # Generar el texto completado basado en el prompt utilizando el modelo
        output = model.generate(input_ids, max_length=50, num_return_sequences=1, early_stopping=True)
        completed_text = tokenizer.decode(output[0], skip_special_tokens=True)

        genres_match = re.search(r'Genres: (.+)', completed_text)
        print("Genres match: ", genres_match)
        if genres_match:
            detected_genres = genres_match.group(1).split(', ')
        else:
            detected_genres = []
        cleaned_genres = [re.sub(r'\s+', ' ', genre.strip()) for genre in detected_genres]
        unique_genres = set(cleaned_genres) #eliminamos generos repetidos
        unique_genres_list = list(unique_genres)
        print("Genres filtered: ", unique_genres_list)
        movies_data.at[index, 'genres_associated'] = unique_genres_list

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Film:  Jupiter's Wife


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 74, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Genres match:  <re.Match object; span=(348, 361), match='Genres: Drama'>
Genres filtered:  ['Drama']
Film:  Desert Winds


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 55, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Genres match:  <re.Match object; span=(354, 367), match='Genres: Drama'>
Genres filtered:  ['Drama']
Film:  Pairon Talle


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Genres match:  <re.Match object; span=(164, 222), match='Genres: Drama, Comedy, Romance, Fantasy, Romance,>
Genres filtered:  ['Fantasy', 'Fantasy,', 'Romance', 'Drama', 'Comedy']
Film:  Grandma Lo-Fi


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 119, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Genres match:  <re.Match object; span=(488, 500), match='Genres: Folk'>
Genres filtered:  ['Folk']
Film:  The Jazz Loft According to W. Eugene Smith
Genres match:  <re.Match object; span=(425, 436), match='Genres: Art'>
Genres filtered:  ['Art']


In [5]:
# Verificar si la columna genres_associated contiene algún valor [] en el DataFrame movies_new
has_empty_lists = movies_data['genres_associated'].apply(lambda x: x == '[]').any()

# Mostrar el resultado
if has_empty_lists:
    print("La columna genres_associated contiene []")
else:
    print("La columna genres_associated no contiene []")

La columna genres_associated no contiene []


In [7]:
movies_data.head(5000).to_csv("/content/drive/MyDrive/dataIA/movies/finalMovies5K.csv", index=False)